# Exercise: Generating one token at a time

In this exercise, we will get to understand how an LLM generates text--one token at a time, using the previous tokens to predict the following ones.

## Step 1. Load a tokenizer and a model

First we load a tokenizer and a model from HuggingFace's transformers library. A tokenizer is a function that splits a string into a list of numbers that the model can understand.

In this exercise, all the code will be written for you. All you need to do is follow along!

In [74]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [75]:
# To load a pretrained model and a tokenizer using Hugging Face, we only need two lines of code.
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

/opt/homebrew/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [76]:
# We create a partial sentence and tokenize it.
text = "Once upon a time, Generative AI models"

inputs = tokenizer(text, return_tensors="pt")

In [77]:
# Show the tokens as numbers, i.e. "input_ids"
print(inputs["input_ids"])

tensor([[7454, 2402,  257,  640,   11, 2980,  876, 9552, 4981]])


In [78]:
inputs

{'input_ids': tensor([[7454, 2402,  257,  640,   11, 2980,  876, 9552, 4981]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}

## Step 2. Examine the tokenization

Let's explore what these tokens mean!

In [79]:
# Show how the sentence is tokenized
import pandas as pd



In [80]:
def show_tokenization(inputs):
    return pd.DataFrame(
        [(id, tokenizer.decode(id)) for id in inputs["input_ids"][0]],
        columns =["id", "token"],
    )
    
    

In [81]:
show_tokenization(inputs)

,id,token
0,tensor(7454),Once
1,tensor(2402),upon
2,tensor(257),a
3,tensor(640),time
4,tensor(11),","
5,tensor(2980),Gener
6,tensor(876),ative
7,tensor(9552),AI
8,tensor(4981),models


### Subword tokenization

The interesting thing is that tokens in this case are neither just letters nor just words. Sometimes shorter words are represented by a single token, but other times a single token represents a part of a word, or even a single letter. This is called subword tokenization.

## Step 2. Calculate the probability of the next token

Now let's use PyTorch to calculate the probability of the next token given the previous ones.

In [82]:
# Calculate the probabilites for the next token for all possible choices. We show the
# top 5 choices and the corresponding words or subwords for these tokens.

import torch 

with torch.no_grad():
    logits = model(**inputs).logits[:, -1, :]
    probabilities = torch.nn.functional.softmax(logits[0], dim=-1)

In [83]:
def show_next_token_choices(probabilities, top_n=5):
    return pd.DataFrame(
        [
           (id, tokenizer.decode(id), p.item())
           for id, p in enumerate(probabilities) 
           if p.item()
        ],
        columns=["id", "token", "p"]
    ).sort_values("p", ascending=False)[:top_n]

In [84]:
show_next_token_choices(probabilities)

,id,token,p
547,547,were,0.198260
550,550,had,0.052242
286,286,of,0.042023
423,423,have,0.039806
389,389,are,0.038839


In [85]:
# Obtain the token id for the most probable next token
next_token_id = torch.argmax(probabilities).item()

In [86]:
print(f"Next token id: {next_token_id}")
print(f"Next token: {tokenizer.decode(next_token_id)}")

Next token id: 547
Next token:  were


In [87]:
#  We append the most likely token to the text. 
text += tokenizer.decode(next_token_id)
text

'Once upon a time, Generative AI models were'

## Step 3. Generate some more tokens

The following cell will take `text`, show the most probable tokens to follow, and append the most likely token to text. Run the cell over and over to see it in action!

In [88]:
# Press ctrl + enter to run the code in a cell
from IPython.display import display, Markdown 

# Show the text
print(text)

Once upon a time, Generative AI models were


In [89]:
# Convert to tokenss  
inputs = tokenizer(text, return_tensors="pt")

In [90]:
# Calculate the probalities for the net toekn and show the top 5 choices
with torch.no_grad():
    logits = model(**inputs).logits[:, -1, :]
    probabilities = torch.nn.functional.softmax(logits[0], dim=-1)

In [91]:
display(Markdown(f"**{text}**"))
show_next_token_choices(probabilities)

**Once upon a time, Generative AI models were**

,id,token,p
973,973,used,0.069505
1498,1498,able,0.030871
1807,1807,thought,0.028160
4166,4166,developed,0.026123
262,262,the,0.022665


## Step 4. Use the `generate` method

In [92]:
from IPython.display import display, Markdown

# Start with some text and tokenize it
text = "Once upon a time, generative models"
inputs = tokenizer(text, return_tensors="pt")

In [93]:
# Use the 'generate' method to generate lots of text
output = model.generate(**inputs, max_length=100, pad_token_id=tokenizer.eos_token_id)

In [94]:
output

tensor([[ 7454,  2402,   257,   640,    11,  1152,   876,  4981,   286,   262,
          1692,  3632,   547,   973,   284,  2050,   262, 17019, 42567,   286,
         10870,  2163,    13,   554,   262,  1944,  2050,    11,   356,   973,
           257,  5337,  2746,   286,   262,  1692,  3632,   284,  9161,   262,
         17019, 42567,   286, 10870,  2163,    13,   775,   973,   257,  5337,
          2746,   286,   262,  1692,  3632,   284,  9161,   262, 17019, 42567,
           286, 10870,  2163,    13,   775,   973,   257,  5337,  2746,   286,
           262,  1692,  3632,   284,  9161,   262, 17019, 42567,   286, 10870,
          2163,    13,   775,   973,   257,  5337,  2746,   286,   262,  1692,
          3632,   284,  9161,   262, 17019, 42567,   286, 10870,  2163,    13]])

In [95]:
# Show the generated text
display(Markdown(tokenizer.decode(output[0])))

Once upon a time, generative models of the human brain were used to study the neural correlates of cognitive function. In the present study, we used a novel model of the human brain to investigate the neural correlates of cognitive function. We used a novel model of the human brain to investigate the neural correlates of cognitive function. We used a novel model of the human brain to investigate the neural correlates of cognitive function. We used a novel model of the human brain to investigate the neural correlates of cognitive function.